In [ ]:
import numpy as np
import pandas as pd
import neurokit2 as nk
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, freqz, tf2zpk
from scipy.fft import fft, fftfreq
import os
import json  # Para guardar listas como strings JSON

datos = pd.read_excel(r'C:\Users\ctama\OneDrive\Desktop\Bioseñales\Proyecto 2\Datos\Datos filtrados.xlsx') # Este es el codigo de pandas para leer la base de datos del proyecto 1

In [30]:
pip install python_speech_features

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5907 sha256=fa0060ef4818f873a6bab754e297853a918d438aab9d6d9c9bbf1df7ed9e13d3
  Stored in directory: c:\users\ctama\appdata\local\pip\cache\wheels\60\90\3c\4b5996a95d363fa14525597a19146a940bec467b44b2a14580
Successfully built python_speech_features
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Datos del proyecto 1

In [20]:
datos

,FileName,Rhythm,Beat,PatientAge,Gender,VentricularRate,AtrialRate,QRSDuration,QTInterval,QTCorrected,RAxis,TAxis,QRSCount,QOnset,QOffset,TOffset,AgeGroup
0,MUSE_20180712_160452_75000,AFIB,LVH,74,MALE,49,57,92,488,440,64,70,7,221,267,465,61-80
1,MUSE_20180118_131905_87000,AFIB,TWC,72,MALE,151,170,84,298,472,-13,176,25,210,252,359,61-80
2,MUSE_20180114_071444_05000,AFIB,TWC,74,MALE,107,108,82,342,456,50,64,18,216,257,387,61-80
3,MUSE_20180114_132956_24000,AFIB,LVHV TWC,68,MALE,95,87,84,334,419,45,18,16,228,270,395,61-80
4,MUSE_20180118_124852_18000,AFIB,STTU,78,MALE,97,153,104,344,436,-29,29,16,221,273,393,61-80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7475,MUSE_20180114_124959_72000,AT,LVHV STDD,77,FEMALE,110,110,106,310,419,43,53,18,219,272,374,61-80
7476,MUSE_20180113_134050_13000,AT,NONE,79,FEMALE,109,109,72,350,471,22,-9,17,225,261,400,61-80
7477,MUSE_20180114_071212_35000,AT,2AVB STTC,75,FEMALE,143,156,74,328,506,56,-13,24,219,256,383,61-80
7478,MUSE_20180115_123937_31000,AT,NONE,63,FEMALE,99,99,76,348,446,53,43,16,225,263,399,61-80


## Codigo Para el metodo de Mfcc

Se importan dos librerías:

-mfcc para calcular los coeficientes cepstrales en las señales (normalmente de voz, pero aquí se usa con ECG).
-skew y kurtosis para calcular estadísticas (asimetría y forma) de las señales.

Consideraciones:
el codigo "ecg = (ecg - np.mean(ecg)) / np.std(ecg)"  normaliza la señal ECG. Más específicamente, realiza lo que se conoce como una estandarización o z-score, que transforma los datos para que tengan:
Media = 0 ; Desviación estándar = 1 evita que la señal ECG puede tener diferentes niveles de voltaje según el paciente, el equipo, o el ruido. Esta normalización elimina esas diferencias de escala tambien
Hace que los MFCC sean más estables porque el cálculo de MFCC está basado en la energía espectral, y si las señales no están en la misma escala, los resultados podrían estar distorsionados o ser inconsistentes ademas permite comparar los MFCC de diferentes pacientes sin que el nivel absoluto de la señal afecte el resultado.


In [ ]:
from python_speech_features import mfcc
from scipy.stats import skew, kurtosis

# === Parámetros MFCC para ECG === Se definen los parámetros del cálculo MFCC. Aunque MFCC es para voz, se testearon los parametros para mantener la "propiedad visual de el ECG".
fs = 500            # Frecuencia de muestreo de las señales ECG (500 Hz)
winlen = 0.2        # Duración de cada ventana para MFCC (0.2 segundos)
winstep = 0.1       # Tiempo de desplazamiento entre ventanas (0.1 segundos)
nfft = 1024         # Número de puntos para la FFT (resolución de frecuencia)
numcep = 9          # Número de coeficientes MFCC que se van a extraer
preemph = 0         # No se aplica preénfasis a la señal

# === Asegurar columnas básicas === Si el DataFrame datos no tiene una columna para guardar los MFCC crudos, se crea con valores vacíos (NaN).
if "mfcc_feat" not in datos.columns:
    datos["mfcc_feat"] = np.nan

# === Crear columnas de estadísticas MFCC === Se asegura que existan las columnas para cada estadística (media, desviación estándar, asimetría y curtosis) de cada uno de los 9 coeficientes MFCC.
for j in range(numcep):
    for stat in ["mean", "std", "skew", "kurt"]:
        col = f"mfcc_{stat}_c{j+1}"
        if col not in datos.columns:
            datos[col] = np.nan

# También se crean columnas para guardar energía total y entropía de la representación MFCC.

for col in ["mfcc_energy", "mfcc_entropy"]:
    if col not in datos.columns:
        datos[col] = np.nan

# === Ruta base === Define la carpeta donde están guardados los archivos de cada uno de los ECG.
directorio_base = r'C:\Users\ctama\OneDrive\Desktop\Bioseñales\Proyecto 2\Datos\ECGDataDenoised'

# === Procesamiento === Recorre cada archivo listado en la columna 'FileName' del DataFrame datos.
for i, nombre_archivo in enumerate(datos['FileName']):
    try:
        ruta = os.path.join(directorio_base, f"{nombre_archivo}.csv")   #Se construye la ruta completa del archivo .csv correspondiente.
        senal = pd.read_csv(ruta, delimiter=',', names=['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']) # Se lee el archivo CSV y se asignan nombres a las columnas según las derivaciones estándar de un ECG.

        # Si la derivación II (que se quiere analizar) no está, se salta ese archivo y se muestra un mensaje.
        if 'II' not in senal.columns:
            print(f"❌ Archivo {nombre_archivo}: columna 'II' no encontrada.")
            continue

        ecg = senal['II'].values
        ecg = (ecg - np.mean(ecg)) / np.std(ecg) # Se extrae la derivación II y se normaliza (media cero, desviación estándar uno) para que todas las señales estén en la misma escala.

        # === MFCC crudo === Aquí se calcula la matriz MFCC: una representación de la señal basada en frecuencia que resume sus características temporales como si fueran notas musicales, solo que en frecuencias para el ECG.
        mfcc_feat = mfcc(
            signal=ecg,
            samplerate=fs,
            winlen=winlen,
            winstep=winstep,
            numcep=numcep,
            nfilt=20,
            nfft=nfft,
            preemph=preemph,
            ceplifter=0,
            appendEnergy=True
        )

        # Guardar coeficientes MFCC crudos
        datos.at[i, 'mfcc_feat'] = json.dumps(mfcc_feat.tolist())

        # === Extraer estadísticas por coeficiente ===  Para cada coeficiente MFCC (del 1 al 9), se calculan 4 estadísticas que resumen su comportamiento a lo largo del tiempo.
        for j in range(numcep):
            coef = mfcc_feat[:, j]
            datos.at[i, f"mfcc_mean_c{j+1}"] = np.mean(coef)
            datos.at[i, f"mfcc_std_c{j+1}"] = np.std(coef)
            datos.at[i, f"mfcc_skew_c{j+1}"] = skew(coef)
            datos.at[i, f"mfcc_kurt_c{j+1}"] = kurtosis(coef)

        # === Características globales === Se calcula la energía total (suma de cuadrados) y la entropía (nivel de desorden) de la señal MFCC. Se le agrega un + 1e-6 para evitar errores con logaritmos de cero.
        energia = np.sum(mfcc_feat**2)
        entropia = -np.sum((mfcc_feat**2) * np.log(mfcc_feat**2 + 1e-6))  # evitar log(0)

        # Se guardan esos dos valores en sus respectivas columnas para el archivo actual.
        datos.at[i, 'mfcc_energy'] = energia
        datos.at[i, 'mfcc_entropy'] = entropia

    except Exception as e: ## Si algo falla (archivo corrupto, sin datos, etc.), se captura el error y se muestra un mensaje sin detener todo el programa.
        print(f"⚠️ Error en {nombre_archivo}: {e}")

C:\Users\ctama\AppData\Local\Temp\ipykernel_29532\3579298792.py:41: RuntimeWarning: invalid value encountered in divide
  ecg = (ecg - np.mean(ecg)) / np.std(ecg)
C:\Users\ctama\AppData\Local\Temp\ipykernel_29532\3579298792.py:41: RuntimeWarning: invalid value encountered in divide
  ecg = (ecg - np.mean(ecg)) / np.std(ecg)
C:\Users\ctama\AppData\Local\Temp\ipykernel_29532\3579298792.py:41: RuntimeWarning: invalid value encountered in divide
  ecg = (ecg - np.mean(ecg)) / np.std(ecg)
C:\Users\ctama\AppData\Local\Temp\ipykernel_29532\3579298792.py:41: RuntimeWarning: invalid value encountered in divide
  ecg = (ecg - np.mean(ecg)) / np.std(ecg)
C:\Users\ctama\AppData\Local\Temp\ipykernel_29532\3579298792.py:41: RuntimeWarning: invalid value encountered in divide
  ecg = (ecg - np.mean(ecg)) / np.std(ecg)
C:\Users\ctama\AppData\Local\Temp\ipykernel_29532\3579298792.py:41: RuntimeWarning: invalid value encountered in divide
  ecg = (ecg - np.mean(ecg)) / np.std(ecg)
C:\Users\ctama\AppData

In [8]:
datos

,FileName,Rhythm,Beat,PatientAge,Gender,VentricularRate,AtrialRate,QRSDuration,QTInterval,QTCorrected,...,mfcc_mean_c12,mfcc_std_c12,mfcc_skew_c12,mfcc_kurt_c12,mfcc_mean_c13,mfcc_std_c13,mfcc_skew_c13,mfcc_kurt_c13,mfcc_energy,mfcc_entropy
0,MUSE_20180712_160452_75000,AFIB,LVH,74,MALE,49,57,92,488,440,...,0.317677,0.304934,-0.363554,0.821946,0.294474,0.274394,-0.401230,0.906553,8742.477472,-29976.562729
1,MUSE_20180118_131905_87000,AFIB,TWC,72,MALE,151,170,84,298,472,...,0.499813,0.475654,0.687231,-0.200141,0.051948,0.300193,0.601651,0.163642,11401.140754,-43937.755827
2,MUSE_20180114_071444_05000,AFIB,TWC,74,MALE,107,108,82,342,456,...,0.313188,0.193084,-0.164045,-0.363562,0.342498,0.142946,-0.295996,-0.181467,11189.783740,-41579.363721
3,MUSE_20180114_132956_24000,AFIB,LVHV TWC,68,MALE,95,87,84,334,419,...,0.349389,0.182630,-0.129309,0.039392,0.305497,0.166178,-1.570059,8.331633,11041.465633,-41342.370863
4,MUSE_20180118_124852_18000,AFIB,STTU,78,MALE,97,153,104,344,436,...,0.221754,0.349224,-0.102692,-0.814647,0.199383,0.316384,-0.242604,-0.308250,12733.285470,-51120.982936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7475,MUSE_20180114_124959_72000,AT,LVHV STDD,77,FEMALE,110,110,106,310,419,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10455.034438,-37477.637627
7476,MUSE_20180113_134050_13000,AT,NONE,79,FEMALE,109,109,72,350,471,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12977.194708,-51225.839072
7477,MUSE_20180114_071212_35000,AT,2AVB STTC,75,FEMALE,143,156,74,328,506,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12893.248858,-49348.939004
7478,MUSE_20180115_123937_31000,AT,NONE,63,FEMALE,99,99,76,348,446,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11206.978237,-41249.345004


In [11]:
datos = pd.read_excel(r'C:\Users\ctama\OneDrive\Desktop\Bioseñales\Proyecto 2\Datos\Datos filtrados.xlsx')

## Para el DCT

En nuestro caso tenemos una señal larga y continua, como es el electrocardiograma (ECG), que registra los latidos del corazón. En lugar de analizar toda la señal de golpe, lo que hacemos es tomar pequeños fragmentos de ella, centrados en los picos R (que marcan cada latido). A estos fragmentos los llamamos ventanas, y nos permiten estudiar cada latido con más detalle, uno por uno, en este caso definimos cada ventana como 500 milisegundos dado que nuestra señal tiene 500 muestras por segundo (fs 500hz), Cuando se aplica la Transformada Discreta del Coseno (DCT), se obtiene una lista de coeficientes que representan las frecuencias dominantes en el latido. los primeros coeficientes capturan la mayor parte de la "energía" del latido, es decir, la información más relevante. El resto suele ser ruido o detalles menos importantes, por eso denominamos que el numero de coeficientes sea 50,

In [ ]:
import pandas as pd
import numpy as np
import os
import json
import neurokit2 as nk
from scipy.fftpack import dct
from scipy.stats import skew, kurtosis

# === Parámetros generales ===
fs = 500                                          # Frecuencia de muestreo del ECG en Hz
ventana_ms = 500                                  # Duración de la ventana para análisis en milisegundos
ventana_muestras = int(fs * (ventana_ms / 1000))  # Convertimos la duración de la ventana a número de muestras
num_coef_dct = 50                                 # Número de coeficientes DCT a extraer por latido

# === Columnas base === Si la columna para guardar los vectores DCT no existe, se crea como NaN
if 'dct_features' not in datos.columns:
    datos['dct_features'] = np.nan

# === Agregar columnas estadísticas DCT === Por cada estadística (media, desviación, asimetría, etc), creamos una columna si aún no existe
for stat in ["mean", "std", "skew", "kurt", "energy", "entropy"]:
    col = f'dct_{stat}'
    if col not in datos.columns:
        datos[col] = np.nan

# === Agregar columnas individuales para coeficientes DCT === También creamos columnas para guardar individualmente cada uno de los coeficientes DCT
for c in range(1, num_coef_dct + 1):
    col = f'dct_coef_c{c}'
    if col not in datos.columns:
        datos[col] = np.nan

# === Ruta ECG === Ruta base donde están almacenados los archivos de señales ECG
directorio_base = r'C:\Users\ctama\OneDrive\Desktop\Bioseñales\Proyecto 2\Datos\ECGDataDenoised'

# === Procesamiento por archivo ===  Recorremos cada archivo de la base de datos
for i, nombre_archivo in enumerate(datos['FileName']):
    try:
        # Cargamos la señal desde archivo CSV
        ruta = os.path.join(directorio_base, f"{nombre_archivo}.csv")
        senal = pd.read_csv(ruta, delimiter=',', names=['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'])

        # Verificamos que exista la derivación II (es la que analizaremos)
        if 'II' not in senal.columns:
            print(f"❌ {nombre_archivo}: columna 'II' no encontrada.")
            continue

        # Extraemos la señal de la derivación II
        ecg = senal['II'].values

        # === Normalización === Centramos la señal alrededor de cero y la escalamos para que tenga desviación estándar 1 Esto evita que la amplitud de la señal afecte el análisis de patrones
        ecg = (ecg - np.mean(ecg)) / np.std(ecg)

        # === Detección de picos R === Usamos NeuroKit para detectar los picos R del ECG, que indican los latidos
        _, info = nk.ecg_peaks(ecg, sampling_rate=fs)
        r_peaks = info["ECG_R_Peaks"]

        dct_vectors = []  # Aquí almacenaremos los vectores DCT de cada latido

        # Recorremos cada pico R detectado
        for r in r_peaks:
            # Definimos la ventana centrada en el pico R
            start = r - ventana_muestras // 2
            end = r + ventana_muestras // 2

            # Si la ventana se sale de los límites de la señal, se descarta ese latido
            if start < 0 or end > len(ecg):
                continue

            latido = ecg[start:end]  # Extraemos la señal del latido

            # === Análisis del latido === Aplicamos autocorrelación para resaltar la estructura interna del latido
            autocor = np.correlate(latido, latido, mode='full')[ventana_muestras:]

            # Aplicamos la Transformada Discreta del Coseno (DCT) sobre la autocorrelación Esto nos da un resumen espectral del latido
            dct_result = dct(autocor, norm='ortho')

            # Guardamos solo los primeros coeficientes
            dct_vectors.append(dct_result[:num_coef_dct])

        # === Si se detectaron latidos válidos ===
        if dct_vectors:
            # Calculamos el promedio de todos los vectores DCT obtenidos
            dct_mean_vector = np.mean(dct_vectors, axis=0)

            # Guardamos el vector promedio completo como JSON
            datos.at[i, 'dct_features'] = json.dumps(dct_mean_vector.tolist())

            # === Guardamos cada coeficiente individual en su columna ===
            for c in range(num_coef_dct):
                datos.at[i, f'dct_coef_c{c+1}'] = dct_mean_vector[c]

            # === Calculamos estadísticas generales del vector promedio ===
            datos.at[i, 'dct_mean'] = np.mean(dct_mean_vector)
            datos.at[i, 'dct_std'] = np.std(dct_mean_vector)
            datos.at[i, 'dct_skew'] = skew(dct_mean_vector)
            datos.at[i, 'dct_kurt'] = kurtosis(dct_mean_vector)

            # === Calculamos energía y entropía del vector ===
            energia = np.sum(dct_mean_vector**2)  # Energía total del vector
            entropia = -np.sum((dct_mean_vector**2) * np.log(dct_mean_vector**2 + 1e-6))  # Entropía espectral
            datos.at[i, 'dct_energy'] = energia
            datos.at[i, 'dct_entropy'] = entropia
        else:
            print(f"⚠️ {nombre_archivo}: sin latidos válidos para análisis.")

    except Exception as e:  # # Si ocurre cualquier error durante el procesamiento del archivo, lo reportamos
        print(f"⚠️ Error en {nombre_archivo}: {e}")

⚠️ MUSE_20180114_070744_27000: sin latidos válidos para análisis.
⚠️ MUSE_20180113_185325_97000: sin latidos válidos para análisis.


C:\Users\ctama\AppData\Local\Temp\ipykernel_29532\791892875.py:45: RuntimeWarning: invalid value encountered in divide
  ecg = (ecg - np.mean(ecg)) / np.std(ecg)


⚠️ MUSE_20181222_204237_47000: sin latidos válidos para análisis.


C:\Users\ctama\AppData\Local\Temp\ipykernel_29532\791892875.py:45: RuntimeWarning: invalid value encountered in divide
  ecg = (ecg - np.mean(ecg)) / np.std(ecg)
c:\Python312\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Python312\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\ctama\AppData\Local\Temp\ipykernel_29532\791892875.py:45: RuntimeWarning: invalid value encountered in divide
  ecg = (ecg - np.mean(ecg)) / np.std(ecg)


⚠️ MUSE_20181222_204122_52000: sin latidos válidos para análisis.
⚠️ MUSE_20180712_150937_19000: sin latidos válidos para análisis.
⚠️ MUSE_20181222_204310_31000: sin latidos válidos para análisis.


...

⚠️ MUSE_20181222_204248_77000: sin latidos válidos para análisis.
⚠️ MUSE_20181222_204226_00000: sin latidos válidos para análisis.
⚠️ MUSE_20181222_204157_58000: sin latidos válidos para análisis.
⚠️ MUSE_20180113_181610_53000: sin latidos válidos para análisis.
⚠️ MUSE_20180113_181614_77000: sin latidos válidos para análisis.
⚠️ MUSE_20181222_204212_44000: sin latidos válidos para análisis.


C:\Users\ctama\AppData\Local\Temp\ipykernel_29532\791892875.py:45: RuntimeWarning: invalid value encountered in divide
  ecg = (ecg - np.mean(ecg)) / np.std(ecg)
C:\Users\ctama\AppData\Local\Temp\ipykernel_29532\791892875.py:45: RuntimeWarning: invalid value encountered in divide
  ecg = (ecg - np.mean(ecg)) / np.std(ecg)


⚠️ MUSE_20181222_204240_84000: sin latidos válidos para análisis.


C:\Users\ctama\AppData\Local\Temp\ipykernel_29532\791892875.py:45: RuntimeWarning: invalid value encountered in divide
  ecg = (ecg - np.mean(ecg)) / np.std(ecg)


⚠️ MUSE_20181222_204123_64000: sin latidos válidos para análisis.
⚠️ MUSE_20180113_181613_71000: sin latidos válidos para análisis.
⚠️ MUSE_20180113_181611_59000: sin latidos válidos para análisis.


In [14]:
datos

,FileName,Rhythm,Beat,PatientAge,Gender,VentricularRate,AtrialRate,QRSDuration,QTInterval,QTCorrected,...,dct_coef_c41,dct_coef_c42,dct_coef_c43,dct_coef_c44,dct_coef_c45,dct_coef_c46,dct_coef_c47,dct_coef_c48,dct_coef_c49,dct_coef_c50
0,MUSE_20180712_160452_75000,AFIB,LVH,74,MALE,49,57,92,488,440,...,2.380878,2.132835,1.977822,2.161554,2.117535,1.835072,1.654479,1.744212,1.604137,1.373413
1,MUSE_20180118_131905_87000,AFIB,TWC,72,MALE,151,170,84,298,472,...,1.224314,0.534052,0.499377,0.879061,0.121342,0.457698,-0.454579,-0.722562,-2.913805,13.961968
2,MUSE_20180114_071444_05000,AFIB,TWC,74,MALE,107,108,82,342,456,...,3.138122,2.565005,2.351326,2.100780,1.774388,1.555822,1.457676,1.353157,1.249000,1.163656
3,MUSE_20180114_132956_24000,AFIB,LVHV TWC,68,MALE,95,87,84,334,419,...,4.775276,3.570046,3.584368,3.514303,2.548161,2.054080,2.023994,1.996300,1.550422,1.383106
4,MUSE_20180118_124852_18000,AFIB,STTU,78,MALE,97,153,104,344,436,...,2.170347,2.020642,2.019786,1.754725,1.485705,1.372253,1.366918,1.313622,1.220172,1.090212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7475,MUSE_20180114_124959_72000,AT,LVHV STDD,77,FEMALE,110,110,106,310,419,...,2.207518,2.019921,1.589326,1.349638,1.243553,1.155440,1.096370,1.107632,0.989800,0.884814
7476,MUSE_20180113_134050_13000,AT,NONE,79,FEMALE,109,109,72,350,471,...,6.222580,5.633672,4.596252,3.218666,2.661873,2.376562,2.163714,1.737861,1.589570,1.435193
7477,MUSE_20180114_071212_35000,AT,2AVB STTC,75,FEMALE,143,156,74,328,506,...,1.380882,1.170363,1.183693,1.190167,0.954719,0.860755,0.838137,0.834799,0.704389,0.676799
7478,MUSE_20180115_123937_31000,AT,NONE,63,FEMALE,99,99,76,348,446,...,1.655274,1.317436,1.256937,1.207338,1.271783,1.015032,0.973202,0.982236,1.044940,0.819897


In [15]:
datos.to_excel('DatosConDcT2.xlsx', index=False, sheet_name='Datos')

## Para el DFT

In [ ]:
# === Crear columnas en el DataFrame === Estas columnas almacenarán las características espectrales extraídas
for col in ["frecuencia_muestreo", "frecuencia_pico", "energia_total", "espectro_magnitud", "fase"]:
    if col not in datos.columns:
        datos[col] = np.nan

# === Parámetros generales ===
fs = 500  # Frecuencia de muestreo en Hz (500 muestras por segundo)
directorio_base = r'C:\Users\ctama\OneDrive\Desktop\Bioseñales\Proyecto 2\Datos\ECGDataDenoised'  # Ruta a los archivos ECG

# === Función para aplicar un filtro pasa banda (elimina ruido fuera de 0.5–40 Hz) ===
def bandpass_filter(signal, fs, lowcut=0.5, highcut=40.0, order=4):
    nyq = 0.5 * fs              # Frecuencia de Nyquist (la máxima frecuencia que se puede medir)
    low = lowcut / nyq          # Normalizamos la frecuencia baja
    high = highcut / nyq        # Normalizamos la frecuencia alta

    # Creamos un filtro de Butterworth de cuarto orden con el rango deseado
    b, a = butter(order, [low, high], btype='band')

    # Aplicamos el filtro en ambas direcciones para evitar desfase
    return filtfilt(b, a, signal)

# === Procesar todos los archivos ECG listados en el DataFrame ===
for i, nombre_archivo in enumerate(datos['FileName']):
    try:
        # Construimos la ruta completa del archivo
        ruta = os.path.join(directorio_base, f"{nombre_archivo}.csv")

        # Cargamos el archivo CSV con los canales estándar del ECG
        senal = pd.read_csv(ruta, delimiter=',', names=['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'])

        # Verificamos que la derivación II esté presente (usualmente es la más usada)
        if 'II' not in senal.columns:
            print(f"❌ Archivo {nombre_archivo}: columna 'II' no encontrada.")
            continue

        # Extraemos la señal de la derivación II como un arreglo de NumPy
        ecg_signal = senal['II'].values

        # Filtramos la señal para eliminar ruido de alta y baja frecuencia
        ecg_filtrada = bandpass_filter(ecg_signal, fs)

        # === Análisis espectral ===

        # Número de muestras
        n = len(ecg_filtrada)

        # Calculamos la FFT (Transformada Rápida de Fourier) para obtener las frecuencias presentes
        yf = fft(ecg_filtrada)  # Resultado complejo: incluye magnitud y fase
        xf = fftfreq(n, 1 / fs)  # Vector de frecuencias correspondientes

        # Nos quedamos solo con la mitad positiva del espectro (la otra mitad es redundante)
        positive_freqs = xf[:n // 2]
        magnitude = np.abs(yf[:n // 2])  # Magnitud del espectro (contenido de frecuencia)
        phase = np.angle(yf[:n // 2])   # Fase del espectro (en radianes)

        # === Características extraídas ===

        # Frecuencia con mayor magnitud: la dominante
        peak_freq = positive_freqs[np.argmax(magnitude)]

        # Energía total del espectro (cuadrado de la magnitud)
        total_energy = np.sum(magnitude ** 2)

        # === Guardar resultados en el DataFrame ===
        datos.at[i, 'frecuencia_muestreo'] = fs
        datos.at[i, 'frecuencia_pico'] = peak_freq
        datos.at[i, 'energia_total'] = total_energy
        datos.at[i, 'espectro_magnitud'] = json.dumps(magnitude.tolist())  # Guardamos la magnitud como lista JSON
        datos.at[i, 'fase'] = json.dumps(phase.tolist())  # Lo mismo con la fase

        # Mensaje de éxito para este archivo
        print(f"✅ {nombre_archivo} procesado: f_pico = {peak_freq:.2f} Hz, E = {total_energy:.2f}")

    except Exception as e:
        # Si ocurre un error, lo reportamos con el nombre del archivo
        print(f"⚠️ Error en {nombre_archivo}: {e}")

C:\Users\ctama\AppData\Local\Temp\ipykernel_79592\1567348518.py:49: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[881.6367081442197, 903.5544332862048, 847.3446365309519, 1268.6815727274027, 4092.5777850012387, 8181.715753135518, 19811.290370445015, 45374.89715153817, 87430.16185816204, 43670.752439971824, 36174.26243988402, 28107.174591910807, 15964.152115424778, 15146.08636291155, 7583.531070555365, 1848.3555476190268, 6360.541570357795, 23314.62319520696, 38820.909147299404, 20328.907134574714, 28430.87556794193, 8271.900085778654, 37757.49317770894, 72513.5902916297, 28219.844383501426, 117641.57732154285, 83487.38141211409, 17381.6863819496, 11253.898489735739, 50844.90055422651, 4749.152209733853, 80647.46424044076, 24737.965224864518, 145316.58726545575, 17529.622403735415, 5837.753788050095, 64215.425112228906, 14605.449875598724, 34234.37402565464, 21554.365005978285, 3427.624236921609, 29850.4

✅ MUSE_20180712_160452_75000 procesado: f_pico = 3.30 Hz, E = 209498525634.14
✅ MUSE_20180118_131905_87000 procesado: f_pico = 2.60 Hz, E = 22862712029.25
✅ MUSE_20180114_071444_05000 procesado: f_pico = 4.40 Hz, E = 163362918183.66
✅ MUSE_20180114_132956_24000 procesado: f_pico = 1.50 Hz, E = 356748680190.51
✅ MUSE_20180118_124852_18000 procesado: f_pico = 6.80 Hz, E = 489510415769.60
✅ MUSE_20180114_073251_45000 procesado: f_pico = 1.90 Hz, E = 125016099754.10
✅ MUSE_20180114_171855_90000 procesado: f_pico = 0.60 Hz, E = 256851100090.74
✅ MUSE_20180118_130902_75000 procesado: f_pico = 0.50 Hz, E = 346365714395.49
✅ MUSE_20180118_133744_75000 procesado: f_pico = 5.00 Hz, E = 174416212345.43
✅ MUSE_20180118_120315_95000 procesado: f_pico = 1.50 Hz, E = 104384191999.24
✅ MUSE_20180116_170450_55000 procesado: f_pico = 2.90 Hz, E = 27262402003.17
✅ MUSE_20180113_130101_56000 procesado: f_pico = 9.50 Hz, E = 153308884346.72
✅ MUSE_20180116_125257_23000 procesado: f_pico = 2.70 Hz, E = 1171

In [26]:
datos

,FileName,Rhythm,Beat,PatientAge,Gender,VentricularRate,AtrialRate,QRSDuration,QTInterval,QTCorrected,...,QRSCount,QOnset,QOffset,TOffset,AgeGroup,frecuencia_muestreo,frecuencia_pico,energia_total,espectro_magnitud,fase
0,MUSE_20180712_160452_75000,AFIB,LVH,74,MALE,49,57,92,488,440,...,7,221,267,465,61-80,500.0,3.3,2.094985e+11,"[881.6367081442197, 903.5544332862048, 847.344...","[-3.141592653589793, -3.1009788528030575, -3.0..."
1,MUSE_20180118_131905_87000,AFIB,TWC,72,MALE,151,170,84,298,472,...,25,210,252,359,61-80,500.0,2.6,2.286271e+10,"[3043.938734634239, 3233.2202631786995, 5116.5...","[-0.0, -0.07889902925952574, -0.20678550638904..."
2,MUSE_20180114_071444_05000,AFIB,TWC,74,MALE,107,108,82,342,456,...,18,216,257,387,61-80,500.0,4.4,1.633629e+11,"[25489.674690524662, 26003.911005820115, 27986...","[-0.0, 0.029107988426804923, 0.051492614021076..."
3,MUSE_20180114_132956_24000,AFIB,LVHV TWC,68,MALE,95,87,84,334,419,...,16,228,270,395,61-80,500.0,1.5,3.567487e+11,"[1042.1920628080993, 1033.7411603780577, 642.8...","[-3.141592653589793, 3.1414971375995244, -2.88..."
4,MUSE_20180118_124852_18000,AFIB,STTU,78,MALE,97,153,104,344,436,...,16,221,273,393,61-80,500.0,6.8,4.895104e+11,"[9977.344321532248, 10184.11599338282, 11154.0...","[-0.0, 0.005706788662632698, -0.00468691823193..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7475,MUSE_20180114_124959_72000,AT,LVHV STDD,77,FEMALE,110,110,106,310,419,...,18,219,272,374,61-80,500.0,3.7,4.270932e+11,"[4918.938153300143, 4951.14248807747, 4435.262...","[-0.0, -0.0008177486598864255, 0.0441533636326..."
7476,MUSE_20180113_134050_13000,AT,NONE,79,FEMALE,109,109,72,350,471,...,17,225,261,400,61-80,500.0,3.6,7.672681e+10,"[20764.638801507186, 21249.402239044255, 24499...","[-3.141592653589793, 3.1290451565351343, 3.083..."
7477,MUSE_20180114_071212_35000,AT,2AVB STTC,75,FEMALE,143,156,74,328,506,...,24,219,256,383,61-80,500.0,2.6,1.146712e+12,"[66010.76808299983, 67308.0374343167, 71925.04...","[-0.0, 0.034364640308522966, 0.064520941475367..."
7478,MUSE_20180115_123937_31000,AT,NONE,63,FEMALE,99,99,76,348,446,...,16,225,263,399,61-80,500.0,6.6,7.232207e+11,"[2650.6757664711854, 2478.0944834756333, 13093...","[-3.141592653589793, -2.3446432077681534, -0.6..."


In [ ]:
datos.to_excel('DatosConDFT.xlsx', index=False, sheet_name='Datos')